
## Assignment-01

### 1. BFS路线搜索


In [1]:
BEIJING, CHANGCHUN, MULUMUQI, WUHAN, GUNAGHZOU, SHENZHEN, BANGKOK, SHANGHAI, NEWYORK = """
BEIJING CHANGCHUN MULUMUQI WUHAN GUANGZHOU SHENZHEN BANGKOK SHANGHAI NEWYORK
""".split()

In [2]:
connection = {
    CHANGCHUN: [BEIJING],
    MULUMUQI: [BEIJING], 
    BEIJING: [MULUMUQI, CHANGCHUN, WUHAN, SHENZHEN, NEWYORK],
    NEWYORK: [BEIJING, SHANGHAI],
    SHANGHAI: [NEWYORK, WUHAN],
    WUHAN: [SHANGHAI, BEIJING, GUNAGHZOU],
    GUNAGHZOU: [WUHAN, BANGKOK],
    SHENZHEN: [WUHAN, BANGKOK],
    BANGKOK: [SHENZHEN, GUNAGHZOU]
}

In [3]:
import networkx as nx
graph = connection
g = nx.Graph(graph)


In [4]:
nx.draw(g)

/Users/heany/miniconda3/lib/python3.7/site-packages/networkx/drawing/nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


In [5]:
def bfs(start):
    queue, path = [] ,[]
    queue.append(start)
    path.append(start)
    while queue:
        v = queue.pop(0)
        for n in connection[v]:
            if n not in path:
                path.append(n)
                queue.append(n)
    return path

In [6]:
bfs(BEIJING)

['BEIJING',
 'MULUMUQI',
 'CHANGCHUN',
 'WUHAN',
 'SHENZHEN',
 'NEWYORK',
 'SHANGHAI',
 'GUANGZHOU',
 'BANGKOK']

### 2.  Sentence Generation

In [7]:
decimal_grammar = """
expression = operator op operator
operator = num op num
num = 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | num num
op = + | - | * | /
"""
grammar = """
sentence => noun_phrase verb_phrase 
noun_phrase => Article Adj* noun
Adj* => null | Adj Adj*
verb_phrase => verb noun_phrase
Article =>  一个 | 这个
noun =>   女人 |  篮球 | 桌子 | 小猫
verb => 看着   |  坐在 |  听着 | 看见
Adj =>   蓝色的 |  好看的 | 小小的
"""

In [8]:
decimal_grammar

'\nexpression = operator op operator\noperator = num op num\nnum = 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | num num\nop = + | - | * | /\n'

In [9]:
grammar

'\nsentence => noun_phrase verb_phrase \nnoun_phrase => Article Adj* noun\nAdj* => null | Adj Adj*\nverb_phrase => verb noun_phrase\nArticle =>  一个 | 这个\nnoun =>   女人 |  篮球 | 桌子 | 小猫\nverb => 看着   |  坐在 |  听着 | 看见\nAdj =>   蓝色的 |  好看的 | 小小的\n'

In [10]:
def parse_grammar(grammar_str, sep="=>"):
    grammar = {}
    for line in grammar_str.split("\n"):
        line = line.strip()
        if  not line: continue
        left, right = line.split(sep)
#         print(right)
        
        grammar[left.strip()] = [item.strip().split() for item in right.split("|")]
    return grammar
    pass

In [11]:
g = parse_grammar(grammar)
g

{'sentence': [['noun_phrase', 'verb_phrase']],
 'noun_phrase': [['Article', 'Adj*', 'noun']],
 'Adj*': [['null'], ['Adj', 'Adj*']],
 'verb_phrase': [['verb', 'noun_phrase']],
 'Article': [['一个'], ['这个']],
 'noun': [['女人'], ['篮球'], ['桌子'], ['小猫']],
 'verb': [['看着'], ['坐在'], ['听着'], ['看见']],
 'Adj': [['蓝色的'], ['好看的'], ['小小的']]}

In [12]:
g["noun"]
g["sentence"]

[['noun_phrase', 'verb_phrase']]

In [13]:
from random import choice
def gene(grammar_ ,target="sentence"):
    if target not in grammar_: return target
    m = choice(grammar_[target])
    
    return "".join(gene(grammar_,target=r) for r in m if r != 'null')
#     return mm
    pass

In [14]:
gene(g)

'这个篮球看着一个小小的好看的桌子'

## 3. 基于模式匹配的对话机器人实现

### Pattern Match
机器能否实现对话，这个长久以来是衡量机器人是否具有智能的一个重要标志。 Alan Turing早在其文中就提出过一个测试机器智能程度的方法，该方法主要是考察人类是否能够通过对话内容区分对方是机器人还是真正的人类，如果人类无法区分，我们就称之为具有”智能“。而这个测试，后来被大家叫做”图灵测试“，之后也被翻拍成了一步著名电影，叫做《模拟游戏》。

既然图灵当年以此作为机器是否具备智能的标志，这项任务肯定是复杂的。自从 1960s 开始，诸多科学家就希望从各个方面来解决这个问题，直到如今，都只能解决一部分问题。 目前对话机器人的建立方法有很多，今天的作业中，我们为大家提供一共快速的基于模板的对话机器人配置方式。

此次作业首先希望大家能够读懂这段程序的代码，其次，在此基于我们提供的代码，能够把它改造成汉语版本，实现对话效果。

    Pattern: (我想要A)
    Response: (如果你有 A，对你意味着什么呢？)

    Input: (我想要度假)
    Response: (如果你有度假，对你意味着什么呢？)

为了实现模板的判断和定义，我们需要定义一个特殊的符号类型，这个符号类型就叫做"variable"， 这个"variable"用来表示是一个占位符。例如，定义一个目标: "I want X"， 我们可以表示成 "I want ?X", 意思就是?X是一个用来占位的符号。

如果输入了"I want holiday"， 在这里 'holiday' 就是 '?X'

In [15]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])
def pat_match(pattern, saying):
    if is_variable(pattern[0]): return True
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])

In [16]:
pat_match('I want ?X'.split(), "I want holiday".split())

True

### 获得匹配的变量
以上的函数能够判断两个 pattern 是不是相符，但是我们更加希望的是获得每个variable对应的是什么值。

我们对程序做如下修改:

In [17]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]):
        return pattern[0], saying[0]
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])
pattern = 'I want ?X'.split()
saying = "I want holiday".split()


In [18]:
pat_match(pattern, saying)

('?X', 'holiday')

In [19]:
pat_match("?X equals ?X".split(), "2+2 equals 2+2".split())

('?X', '2+2')

#### 无法解决具有多个Pattern变量

In [20]:
def pat_match(pattern, saying):
    if not pattern or not saying: return []
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])


In [21]:
pat_match("?X equals ?X".split(), "2+2 equals 2+2".split())

[('?X', '2+2'), ('?X', '2+2')]

如果我们知道了每个变量对应的是什么，那么我们就可以很方便的使用我们定义好的模板进行替换：

为了方便接下来的替换工作，我们新建立两个函数，一个是把我们解析出来的结果变成一个 dictionary，一个是依据这个 dictionary 依照我们的定义的方式进行替换。

In [22]:
def pat_to_dict(patterns):
    return {k: v for k, v in patterns}

In [23]:
pat_to_dict([('?X', '2+2'), ('?z', '2+2')])

{'?X': '2+2', '?z': '2+2'}

In [24]:
# def subsitite(rule, parsed_rules):
#     if not rule: return []
    
#     return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)
# 非递归
def subsitite(rule, parsed_rules):
    if not rule: return []
    sub = rule.copy()
    for r in rule:
        if r in parsed_rules.keys():
            index = sub.index(r)
            sub[index] = parsed_rules[r]
            
#         sub.append(r)
    return sub

In [25]:
got_patterns = pat_match("I want ?X".split(), "I want iPhone".split())
got_patterns

[('?X', 'iPhone')]

In [26]:
res = subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns))
res

['What', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iPhone']

In [27]:
john_pat = pat_match('?P needs ?X'.split(), "John needs vacation".split())
john_pat

[('?P', 'John'), ('?X', 'vacation')]

In [28]:
pat_to_dict(john_pat)

{'?P': 'John', '?X': 'vacation'}

In [29]:
subsitite("Why does ?P need ?X ?".split(), pat_to_dict(john_pat))

['Why', 'does', 'John', 'need', 'vacation', '?']

In [30]:
' '.join(subsitite("Why does ?P need ?X ?".split(), pat_to_dict(john_pat)))

'Why does John need vacation ?'

#### 那么如果我们现在定义一些patterns，就可以实现基于模板的对话生成了:

In [31]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [32]:
def get_pat_match(saying, defined_patterns):
    for p in defined_patterns.keys():
        patters = pat_match(p.split(),saying.split())
        if  patters:
            return p, patters
#     return patters
    pass

In [33]:
import random
def get_response(saying, rules):
    """" please implement the code, to get the response as followings:
    
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    p,patters = get_pat_match(saying, rules)
    patters = pat_to_dict(patters)
    response = random.choice(rules[p])
    return ' '.join(subsitite(response.split(), patters))
    pass

In [34]:
get_response('I need iPhone',defined_patterns)

'Image you will get iPhone soon'

In [35]:
get_response('My mother told me something',defined_patterns)

'How do you think about your mother ?'

####  Segment Match
我们上边的这种形式，能够进行一些初级的对话了，但是我们的模式逐字逐句匹配的， "I need iPhone" 和 "I need ?X" 可以匹配，但是"I need an iPhone" 和 "I need ?X" 就不匹配了，那怎么办？

为了解决这个问题，我们可以新建一个变量类型 "?*X", 这种类型多了一个星号(*),表示匹配多个

首先，和前文类似，我们需要定义一个判断是不是匹配多个的variable

In [36]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])
is_pattern_segment('?*Pb')

True

In [37]:
from collections import defaultdict
fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail
#这段程序里比较重要的一个新函数是 segment_match，这个函数输入是一个以 segment_pattern开头的模式，尽最大可能进行，
#匹配到这个边长的变量对于的部分。
def pat_match(pattern,saying):
    if not pattern or not saying: return []
    
    if is_variable(pattern[0]):
        return [segment_match(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])

def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying),len(saying)  
    
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]),i
    
    return (seg_pat, saying),len(saying)

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

In [38]:
segment_match('?*x hello ?*y'.split(), "li hello si".split())

(('?x', ['li']), 1)

In [39]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

In [40]:
response_pair = {
    'I need ?X': [
        "Why do you neeed ?X"
    ],
    "I dont like my ?X": ["What bad things did ?X do for you?"]
}

In [41]:
pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())

[('?X', ['an', 'iPhone'])]

In [42]:
subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', ['an', 'iPhone']]

In [43]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [44]:
subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', 'an iPhone']

In [45]:
("?*X hello ?*Y", "Hi, how do you do")

('?*X hello ?*Y', 'Hi, how do you do')

In [46]:
subsitite("Hi, how do you do?".split(), pat_to_dict(pat_match_with_seg('?*X hello ?*Y'.split(), 
                  "I am mike, hello ".split())))

['Hi,', 'how', 'do', 'you', 'do?']

## GET_Response

### 问题1

In [47]:
rules = {
    "hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

In [48]:
def get_pat_match(saying, defined_patterns):
    for p in defined_patterns.keys():
        patters = pat_match_with_seg(p.split(),saying.split())
        if  patters != [True, None]:
            return p, patters
#     return patters
    pass

In [49]:
pat_match_with_seg("I was ?*X".split(), 
                  "you bie god b".split())

[True, None]

In [50]:
import random
def get_response(saying, rules):
    """" please implement the code, to get the response as followings:
    
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    p,patters = get_pat_match(saying, rules)
    patters = pat_to_dict(patters)
    response = random.choice(rules[p])
    return ' '.join(subsitite(response.split(), patters))
    pass

In [51]:
get_response('hello zhangsan',rules)

'Hi, how do you do?'

In [52]:
get_response('I was a good boy',rules)

'Were you really a good boy ?'

### 问题2

In [72]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}

#### 分词，处理句子

In [54]:
def is_chinese(uchar):
    if uchar >= u'\u4e00' and uchar <= u'\u9fa5':
        return True
    else:
        return False

In [55]:
def is_alphabet(uchar):
    if len(uchar) >1:
        return False
    if (uchar >= u'\u0041' and uchar <= u'\u005a') or (uchar >= u'\u0061' and uchar <= u'\u007a'):
        return True
    else:
        return False

In [168]:
is_alphabet('a')

True

In [57]:
is_chinese("*")

False

In [150]:
import jieba
jieba.add_word("AI")
jieba.add_word('就像')
jieba.add_word('有的人')
def sen2word(sentence, direction="left"):
#     sent = sentence.replace('*','')
    sent = jieba.lcut(sentence)
    sent = list(filter(lambda x:x!=' ',sent))
    words = []
#     words.append(''.join(sent[0:2]))
    tem = sent[:]
    if direction == "left":
        if '?' in tem:
            for i in range(len(tem)):
                if i+2 < len(tem):
                    if tem[i] == '?' and tem[i+1] == '*' and is_alphabet(tem[i+2]):
                        t = ''.join(tem[i:i+3])
                        words.append(t)
                        tem[i+1] =''
                        tem[i+2] =''
                    else:
                        words.append(tem[i])
                else:
                    words.append(tem[i])
        else:
            for z in tem:
                words.append(z)
    if direction == "right":
        if '?' in tem:
            for i in range(len(tem)):
                if i+1 < len(tem):
                    if tem[i] == '?'  and is_alphabet(tem[i+1]):
                        t = ''.join(tem[i:i+2])
                        words.append(t)
                        tem[i+1] =''
#                         tem[i+2] =''
                    else:
                        words.append(tem[i])
                else:
                    words.append(tem[i])
        else:
            for z in tem:
                words.append(z)
#     words.append(''.join(sent[-3:]))
    
    return list(filter(lambda x : x!='',words))


In [152]:
sen2word('你觉得他们可能不是?*y？')

['你', '觉得', '他们', '可能', '不是', '?*y', '？']

In [154]:
sen2word('你觉得他们可能不是?y？','right')

['你', '觉得', '他们', '可能', '不是', '?y', '？']

In [151]:
[sen2word(pa) for pa in rule_responses.keys()]

[['?*x', 'hello', '?*y'],
 ['?*x', 'I', 'want', '?*y'],
 ['?*x', 'if', '?*y'],
 ['?*x', 'no', '?*y'],
 ['?*x', 'I', 'was', '?*y'],
 ['?*x', 'I', 'feel', '?*y'],
 ['?*x', '你好', '?*y'],
 ['?*x', '我', '想', '?*y'],
 ['?*x', '我', '想要', '?*y'],
 ['?*x', '喜欢', '?*y'],
 ['?*x', '讨厌', '?*y'],
 ['?*x', 'AI', '?*y'],
 ['?*x', '机器人', '?*y'],
 ['?*x', '对不起', '?*y'],
 ['?*x', '我', '记得', '?*y'],
 ['?*x', '如果', '?*y'],
 ['?*x', '我', '?*z', '梦见', '?*y'],
 ['?*x', '妈妈', '?*y'],
 ['?*x', '爸爸', '?*y'],
 ['?*x', '我', '愿意', '?*y'],
 ['?*x', '我', '很', '难过', '，', '因为', '?*y'],
 ['?*x', '难过', '?*y'],
 ['?*x', '就像', '?*y'],
 ['?*x', '和', '?*y', '都', '?*z'],
 ['?*x', '和', '?*y', '一样', '?*z'],
 ['?*x', '我', '是', '?*y'],
 ['?*x', '我', '是', '?*y', '吗'],
 ['?*x', '你', '是', '?*y', '吗'],
 ['?*x', '你', '是', '?*y'],
 ['?*x', '因为', '?*y'],
 ['?*x', '我', '不能', '?*y'],
 ['?*x', '我', '觉得', '?*y'],
 ['?*x', '我', '?*y', '你', '?*z'],
 ['?*x', '你', '为什么', '不', '?*y'],
 ['?*x', '好', '的', '?*y'],
 ['?*x', '嗯', '嗯', '?*y'],
 ['?*x

In [129]:
from collections import defaultdict
fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        if index == len(saying) and len(pattern)>1:
            return fail
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail
# 匹配多个模式
def pat_match(pattern,saying):
    if not pattern or not saying: return []
#     print(pattern[0])
    
    if is_variable(pattern[0]):
        tp,i = segment_match(pattern, saying)
        return [tp] + pat_match(pattern[1:], saying[i:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])
# 匹配最长模式
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying),len(saying)  
    
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]),i  
    return (seg_pat, saying), len(saying)

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v,list) else v for k, v in patterns}

def get_pat_match(saying, defined_patterns):
    for p in defined_patterns.keys():
#         print(sen2word(p))
        patters = pat_match_with_seg(sen2word(p), sen2word(saying))
#         print(patters)
        if  patters != [True,None] and None not in patters:
            return p, patters
#     return patters
    pass

In [116]:
pat_match_with_seg('?*x hello ?*y'.split(), sen2word("张三你好李四"))

[True, None]

In [127]:
pat_match_with_seg(sen2word('?*x他们是?*y吗？'), sen2word("张三他们是李四吗？"))

[('?x', ['张三']), ('?y', ['李四'])]

In [110]:
sen2word("张三我是李四")

['张三', '我', '是', '李四']

In [134]:
get_pat_match('张三他们是李四吗？',rule_responses)

('?*x他们是?*y吗？', [('?x', ['张三']), ('?y', ['李四'])])

In [135]:
p,pp = get_pat_match("张三他们是李四吗？",rule_responses)
print(p,pp)

?*x他们是?*y吗？ [('?x', ['张三']), ('?y', ['李四'])]


In [175]:
import random
def get_response(saying, rules):
    """" please implement the code, to get the response as followings:
    
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    p,patters = get_pat_match(saying, rules)
    patters = pat_to_dict(patters)
#     print(p)
#     print(patters)
    response = random.choice(rules[p])
#     print(response)
    if is_alphabet(patters['?x'][0]):  # 处理英文回答
        return ' '.join(subsitite(sen2word(response,'right'), patters))
    else:  # 中文回答
        return ''.join(subsitite(sen2word(response,'right'), patters))
    pass


In [176]:
get_response('xiao hello ming',rule_responses)

'Please state your problem'

In [177]:
get_response('你最好不要这样',rule_responses)

'为什么不？'

In [178]:
get_response('他好的李四',rule_responses)

'好的'

In [179]:
get_response("张三他们是李四吗？",rule_responses)

'你觉得他们可能不是李四？'

#### 问题3 

暂无

#### 问题 4

1. 这样的程序有什么优缺点？你有什么可以改进的方法吗？

 + 优点：思路简洁明了，
 + 缺点：问答过于僵硬，太过’机器化‘
 + 改进：借助大数据，训练模型进行人机对话

2. 什么是数据驱动？数据驱动在这个程序里如何体现？

 + 数据驱动：数据驱动是指基于大数据分析所做出的决策。
 + 体现：没感觉到

3. 数据驱动与AI的关系是什么？
 
 AI需要大量的数据进行训练出模型，才能做出较为正确的决策，数据驱动是AI的前提，要想实现AI，必须以大数据为基础